# 1. Import library

In [7]:
import pandas as pd

# 2. Read and clean json data

In [8]:
from JsonProcess import JsonProcess
jsonProcesser = JsonProcess(vector2Regex = r"X:|Y:|,Z:.*", vector3Regex = r"X:|Y:|Z:", vector4Regex = r"X:|Y:|Z:|W:")

rawDataPath = "D:/ASD/【Vive】特徵計算/Pre"
jointDataName = "JointTrackingData"

def cleanJointTrackingData(jointData):
    cleanCols = ["Timestamp"]
    cleanJointData = jointData[cleanCols]

    cleanQua = jsonProcesser.splitToVector4(jointData['qua'])
    cleanPos = jsonProcesser.splitToVector3(jointData['pos'])

    cleanJointData = pd.concat([cleanJointData, cleanQua, cleanPos], axis=1)
    

    return cleanJointData

In [9]:
folderPath = f"{rawDataPath}/ASD001_Pre/"
jointData = jsonProcesser.readJsonDataFrame(folderPath, jointDataName)
jointData = cleanJointTrackingData(jointData)
jointData

,Timestamp,qua_X,qua_Y,qua_Z,qua_W,pos_X,pos_Y,pos_Z
0,2022-12-25T11:08:20.7550137+08:00,-0.580724,0.378350,0.622059,-0.364217,2.187241,2.756504,6.316399
1,2022-12-25T11:08:20.951502+08:00,-0.409518,0.374138,0.624035,-0.550361,2.156476,2.726732,6.351755
2,2022-12-25T11:08:20.9764354+08:00,-0.410350,0.410291,0.592654,-0.558601,2.163735,2.720846,6.356529
3,2022-12-25T11:08:20.987406+08:00,-0.410618,0.414495,0.588733,-0.559446,2.164569,2.720178,6.356781
4,2022-12-25T11:08:20.9993737+08:00,-0.410278,0.418835,0.585013,-0.560365,2.165184,2.719384,6.356846
...,...,...,...,...,...,...,...,...
50509,2022-12-25T11:17:59.9556219+08:00,-0.579667,0.457797,0.441417,-0.509469,2.216181,2.669548,6.343642
50510,2022-12-25T11:17:59.9665932+08:00,-0.578839,0.457684,0.443163,-0.508996,2.216323,2.669511,6.343152
50511,2022-12-25T11:17:59.9775644+08:00,-0.577858,0.457688,0.445010,-0.508495,2.216517,2.669494,6.342895
50512,2022-12-25T11:17:59.9905293+08:00,-0.576744,0.457542,0.447241,-0.507935,2.216560,2.669572,6.342783


# 3. Feature calculation
- Features you need should put in the DataFrame named **result**

In [10]:
from FeatureCalculator import HeadMovement
HM_calculator = HeadMovement()

In [11]:
def calculateFeatures(data):
    result = pd.DataFrame()

    vectors = ["X", "Y", "Z"]
    for vector in vectors:
        headAmplitude = HM_calculator.getHeadAmplitude(data, axisColumnName=f"pos_{vector}",prefix=f"{vector}_")
  
        result = pd.concat([result, headAmplitude], axis = 1)

    return result

In [12]:
result = calculateFeatures(jointData)
result

,X_HeadAmplitude,Y_HeadAmplitude,Z_HeadAmplitude
0,27.50355,19.003619,41.075559


In [13]:
x_values = result["X_HeadAmplitude"]
y_values = result["Y_HeadAmplitude"]
z_values = result["Z_HeadAmplitude"]

import math

def distance(x, y, z):
    # 計算三個軸上的移動距離的平方和
    distance_squared = x**2 + y**2 + z**2
    # 取平方根得到真正的移動距離
    distance = math.sqrt(distance_squared)
    return distance

print(distance(x_values, y_values, z_values))


52.960215122924005
